In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx
/kaggle/input/ey-geographical-shape-kml/Building_Footprint.kml
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.shp
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.shx
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.dbf
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.shp.xml
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.cpg
/kaggle/input/ey-geographical-shape-kml/BUILDING_view_-5453825549351750874/BUILDING.prj
/kaggle/input/uhi-index-data/Training_data_uhi_index_UHI2025-v2.csv
/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv


In [2]:
weather_data = pd.read_excel('/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx')
train_df = pd.read_csv('/kaggle/input/uhi-index-data/Training_data_uhi_index_UHI2025-v2.csv')
test_df = pd.read_csv('/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv')

In [3]:
bronx_df = pd.read_excel('/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx', sheet_name='Bronx')
manhattan_df = pd.read_excel('/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx', sheet_name='Manhattan')

In [4]:
bronx_df['Date / Time'] = pd.to_datetime(bronx_df['Date / Time'])
bronx_filtered = bronx_df[(bronx_df['Date / Time'].dt.hour <= 13)]
bronx_filtered

,Date / Time,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,2021-07-24 06:00:00,19.3,88.2,0.8,335,12
1,2021-07-24 06:05:00,19.4,87.9,0.8,329,18
2,2021-07-24 06:10:00,19.3,87.6,0.7,321,25
3,2021-07-24 06:15:00,19.4,87.4,0.5,307,33
4,2021-07-24 06:20:00,19.4,87.0,0.2,301,42
...,...,...,...,...,...,...
91,2021-07-24 13:35:00,26.9,46.6,1.4,110,392
92,2021-07-24 13:40:00,26.8,46.2,2.8,82,532
93,2021-07-24 13:45:00,27.4,46.0,2.9,104,670
94,2021-07-24 13:50:00,27.4,44.7,3.2,102,825


In [5]:
bronx_column_names = {
    bronx_df.columns[1]: 'surface_air_temp',
    bronx_df.columns[2]: 'relative_humidity',
    bronx_df.columns[3]: 'avg_wind_speed',
    bronx_df.columns[4]: 'wind_direction',
    bronx_df.columns[5]: 'solar_flux'
}
manhattan_column_names = {
    manhattan_df.columns[1]: 'surface_air_temp',
    manhattan_df.columns[2]: 'relative_humidity',
    manhattan_df.columns[3]: 'avg_wind_speed',
    manhattan_df.columns[4]: 'wind_direction',
    manhattan_df.columns[5]: 'solar_flux'
}
bronx_df.rename(columns=bronx_column_names, inplace=True)
manhattan_df.rename(columns=manhattan_column_names, inplace=True)

In [6]:
bronx_df.nunique()

Date / Time          169
surface_air_temp      70
relative_humidity    112
avg_wind_speed        44
wind_direction       113
solar_flux           158
dtype: int64

In [7]:
manhattan_df.nunique()

Date / Time          169
surface_air_temp      59
relative_humidity    104
avg_wind_speed        33
wind_direction       119
solar_flux           159
dtype: int64

In [8]:
train_df['datetime'] = pd.to_datetime(train_df['datetime'])
train_df['time'] = train_df['datetime'].dt.time

In [9]:
def time_compress(df):
    # Convert the "Date / Time" column to datetime (if not already)
    df["Date / Time"] = pd.to_datetime(df["Date / Time"])
    
    # Calculate decimal hours (include seconds if available)
    df["time_hours"] = (
        df["Date / Time"].dt.hour + 
        df["Date / Time"].dt.minute / 60.0 + 
        df["Date / Time"].dt.second / 3600.0
    )
    
    # Normalize the original time range (6 to 20) to a 0-1 scale
    df["normalized"] = (df["time_hours"] - 6) / (20 - 6)
    
    # Map the normalized values to the new time range (3 to 4)
    df["new_time_hours"] = 3 + df["normalized"] * (4 - 3)
    
    # Create the new timestamp by normalizing the date (midnight) and adding the new time as a timedelta.
    # We do NOT round here so we preserve the differences (i.e. 21.4 sec differences should stick)
    df["Date / Time"] = df["Date / Time"].dt.normalize() + pd.to_timedelta(df["new_time_hours"], unit="h")
    
    # Drop the intermediate columns
    df.drop(columns=["time_hours", "normalized", "new_time_hours"], inplace=True)
    
    return df


In [10]:
bronx_df = time_compress(bronx_df)
manhattan_df = time_compress(manhattan_df)

In [11]:
df_filtered = bronx_df[(bronx_df['Date / Time'].dt.hour >= 2) & (bronx_df['Date / Time'].dt.hour < 3)]
df_filtered

,Date / Time,surface_air_temp,relative_humidity,avg_wind_speed,wind_direction,solar_flux


In [12]:
# weather_path = '/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx'
# def load_weather_station(xlsx_path):
#     bronx_df = pd.read_excel(xlsx_path, sheet_name='Bronx')
#     manhattan_df = pd.read_excel(xlsx_path, sheet_name='Manhattan')
#     bronx_df['Date / Time'] = pd.to_datetime(bronx_df['Date / Time'])
#     manhattan_df['Date / Time'] = pd.to_datetime(manhattan_df['Date / Time'])
#     bronx_filtered = bronx_df[(bronx_df['Date / Time'].dt.hour >= 14) & (bronx_df['Date / Time'].dt.hour < 17)]
#     manhattan_filtered = manhattan_df[(bronx_df['Date / Time'].dt.hour >= 14) & (manhattan_df['Date / Time'].dt.hour < 17)]
#     bronx_df = time_compress(bronx_filtered)
#     manhattan_df = time_compress(manhattan_filtered)
#     # bronx_df['Date / Time'] = pd.to_datetime(bronx_df['Date / Time'])
#     # manhattan_df['Date / Time'] = pd.to_datetime(manhattan_df['Date / Time'])
#     # bronx_column_names = {
#     # bronx_df.columns[1]: 'surface_air_temp',
#     # bronx_df.columns[2]: 'relative_humidity',
#     # bronx_df.columns[3]: 'avg_wind_speed',
#     # bronx_df.columns[4]: 'wind_direction',
#     # bronx_df.columns[5]: 'solar_flux'
#     # }
#     # manhattan_column_names = {
#     # manhattan_df.columns[1]: 'surface_air_temp',
#     # manhattan_df.columns[2]: 'relative_humidity',
#     # manhattan_df.columns[3]: 'avg_wind_speed',
#     # manhattan_df.columns[4]: 'wind_direction',
#     # manhattan_df.columns[5]: 'solar_flux'
#     # }
#     # bronx_df.rename(columns=bronx_column_names, inplace=True)
#     # manhattan_df.rename(columns=manhattan_column_names, inplace=True)
#     bronx_df.sort_values('Date / Time', inplace=True)
#     manhattan_df.sort_values('Date / Time', inplace=True)
#     return bronx_df, manhattan_df

# def pick_station(lat, lon):

#     if 40.80 <= lat <= 40.92 and -73.93 <= lon <= -73.79:
#         return "Bronx"
#     # elif 40.70 <= lat <= 40.88 and -74.02 <= lon <= -73.91:
#     #     return "Manhattan"
#     else: 
#         return "Manhattan"

# def match_nearest_weather(row_dt, weather_df):
#     print()
#     diffs = (weather_df['Date / Time'] - row_dt).abs()
#     idx_min = diffs.idxmin()
#     return weather_df.loc[idx_min]

# def add_weather_to_train(train_df, bronx_df, manh_df):

#     weather_cols = ["Air Temp at Surface [degC]", "Relative Humidity [percent]",
#                     "Avg Wind Speed [m/s]", "Wind Direction [degrees]", "Solar Flux [W/m^2]"]
#     at_surface = []
#     rh = []
#     wind_speed = []
#     wind_dir = []
#     solar_flux = []

#     for _, row in train_df.iterrows():

#         stn = pick_station(row['Latitude'], row['Longitude'])
#         if stn == 'Bronx':
#             wdf = bronx_df
#         else:
#             wdf = manh_df

#         row_dt = pd.to_datetime(row['datetime'])
#         match = match_nearest_weather(row_dt, wdf)

#         at_surface.append(match["Air Temp at Surface [degC]"])
#         rh.append(match["Relative Humidity [percent]"])
#         wind_speed.append(match["Avg Wind Speed [m/s]"])
#         wind_dir.append(match["Wind Direction [degrees]"])
#         solar_flux.append(match["Solar Flux [W/m^2]"])

#     train_df["Weather_AirTemp"] = at_surface
#     train_df["Weather_RH"]      = rh
#     train_df["Weather_WindSpd"] = wind_speed
#     train_df["Weather_WindDir"] = wind_dir
#     train_df["Weather_Solar"]   = solar_flux

#     return train_df

# weather_bronx, weather_manh = load_weather_station(weather_path)
# train_enh_1 = add_weather_to_train(train_df, weather_bronx, weather_manh)

In [13]:
import pandas as pd
import numpy as np

weather_path = '/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx'

def load_weather_station(xlsx_path, upper, lower):
    """Load weather data and compress time from 2–5 PM to 3–4 PM"""
    bronx_df = pd.read_excel(xlsx_path, sheet_name='Bronx')
    manhattan_df = pd.read_excel(xlsx_path, sheet_name='Manhattan')
    
    def process_and_compress(df):
        df['Date / Time'] = pd.to_datetime(df['Date / Time'])
        # Filter to original window 2–5 PM (i.e. 14:00 to 17:00)
        df['hour'] = df['Date / Time'].dt.hour
        df = df[(df['hour'] >= 14) & (df['hour'] < 17)]
        # Convert time to decimal hours
        df['time_hours'] = (df['Date / Time'].dt.hour +
                            df['Date / Time'].dt.minute/60.0 +
                            df['Date / Time'].dt.second/3600.0)
        # Linear compression from original [14,17) to new [15,16)
        orig_lower, orig_upper = lower, upper
        new_lower, new_upper = 15, 16
        df['compressed_time'] = new_lower + (df['time_hours'] - orig_lower) * ((new_upper - new_lower) / (orig_upper - orig_lower))
        # Replace Date / Time with normalized date plus compressed time
        df['Date / Time'] = df['Date / Time'].dt.normalize() + pd.to_timedelta(df['compressed_time'], unit='h')
        df.sort_values('Date / Time', inplace=True)
        df.drop(columns=['hour', 'time_hours', 'compressed_time'], inplace=True)
        return df

    bronx_df = process_and_compress(bronx_df)
    manhattan_df = process_and_compress(manhattan_df)
    return bronx_df, manhattan_df

def pick_station(lat, lon):
    if 40.80 <= lat <= 40.92 and -73.93 <= lon <= -73.79:
        return "Bronx"
    else:
        return "Manhattan"

def get_closest_weather(target_dt, weather_df):
    """Return weather features from the row with the closest datetime"""
    same_day = weather_df[weather_df['Date / Time'].dt.date == target_dt.date()]
    if same_day.empty:
        return None
    diffs = (same_day['Date / Time'] - target_dt).abs()
    idx_min = diffs.idxmin()
    row = same_day.loc[idx_min]
    return {
        'AirTemp': row['Air Temp at Surface [degC]'],
        'RH': row['Relative Humidity [percent]'],
        'WindSpd': row['Avg Wind Speed [m/s]'],
        'WindDir': row['Wind Direction [degrees]'],
        'Solar': row['Solar Flux [W/m^2]']
    }

def add_weather_to_train(train_df, bronx_df, manh_df):
    """Match each train row to the closest weather row (after compression)"""
    bronx_dates = {d: g for d, g in bronx_df.groupby(bronx_df['Date / Time'].dt.date)}
    manh_dates = {d: g for d, g in manh_df.groupby(manh_df['Date / Time'].dt.date)}
    
    weather_cols = ['Weather_AirTemp', 'Weather_RH', 'Weather_WindSpd', 'Weather_WindDir', 'Weather_Solar']
    for col in weather_cols:
        train_df[col] = np.nan
    
    train_df['datetime'] = pd.to_datetime(train_df['datetime'])
    for idx, row in train_df.iterrows():
        stn = pick_station(row['Latitude'], row['Longitude'])
        target_date = row['datetime'].date()
        weather_data = bronx_dates.get(target_date) if stn == 'Bronx' else manh_dates.get(target_date)
        if weather_data is not None:
            features = get_closest_weather(row['datetime'], weather_data)
            if features is not None:
                train_df.at[idx, 'Weather_AirTemp'] = features['AirTemp']
                train_df.at[idx, 'Weather_RH'] = features['RH']
                train_df.at[idx, 'Weather_WindSpd'] = features['WindSpd']
                train_df.at[idx, 'Weather_WindDir'] = features['WindDir']
                train_df.at[idx, 'Weather_Solar'] = features['Solar']
    return train_df

# Usage



In [14]:
# weather_bronx, weather_manh = load_weather_station(weather_path, 12, 17)
# train_enh_12_17 = add_weather_to_train(train_df.copy(), weather_bronx, weather_manh)
# train_enh_12_17.to_csv('train_enh_12_17.csv', index=False)

In [15]:
# weather_bronx, weather_manh = load_weather_station(weather_path, 13, 17)
# train_enh_13_17 = add_weather_to_train(train_df.copy(), weather_bronx, weather_manh)
# train_enh_13_17.to_csv('train_enh_13_17.csv', index=False)

In [16]:
# weather_bronx, weather_manh = load_weather_station(weather_path, 14, 18)
# train_enh_14_18 = add_weather_to_train(train_df.copy(), weather_bronx, weather_manh)
# train_enh_14_18.to_csv('train_enh_14_18.csv', index=False)

In [17]:
# weather_bronx, weather_manh = load_weather_station(weather_path, 14, 19)
# train_enh_14_19 = add_weather_to_train(train_df.copy(), weather_bronx, weather_manh)
# train_enh_14_19.to_csv('train_enh_14_19.csv', index=False)

In [18]:
# weather_bronx, weather_manh = load_weather_station(weather_path, 14, 20)
# train_enh_14_20 = add_weather_to_train(train_df.copy(), weather_bronx, weather_manh)
# train_enh_14_20.to_csv('train_enh_14_20.csv', index=False)

In [19]:
# weather_bronx, weather_manh = load_weather_station(weather_path, 6, 20)
# train_enh_6_20 = add_weather_to_train(train_df.copy(), weather_bronx, weather_manh)
# train_enh_6_20.to_csv('train_enh_6_20.csv', index=False)

In [20]:
# weather_bronx, weather_manh = load_weather_station(weather_path, 11, 20)
# train_enh_11_20 = add_weather_to_train(train_df.copy(), weather_bronx, weather_manh)
# train_enh_11_20.to_csv('train_enh_14_20.csv', index=False)

# Approach_2

In [21]:
# def normalize_time_window(datetime, target_start_hour=3, target_end_hour=4):
#     """
#     Normalize input datetime to a target time window
    
#     Args:
#     - datetime: Input datetime
#     - target_start_hour: Start of desired time window (default 3)
#     - target_end_hour: End of desired time window (default 4)
    
#     Returns:
#     - Normalized datetime within target window
#     """
#     hour = datetime.hour
    
#     # If hour is already in target window, return original datetime
#     if target_start_hour <= hour < target_end_hour:
#         return datetime
    
#     # Adjust datetime to nearest target window
#     if hour < target_start_hour:
#         # Earlier in the day, shift to target start hour
#         normalized_datetime = datetime.replace(hour=target_start_hour, minute=0, second=0, microsecond=0)
#     else:
#         # Later in the day, shift to next day's target start hour
#         normalized_datetime = (datetime + pd.Timedelta(days=1)).replace(hour=target_start_hour, minute=0, second=0, microsecond=0)
    
#     return normalized_datetime

# def load_weather_station(xlsx_path):
#     bronx_df = pd.read_excel(xlsx_path, sheet_name='Bronx')
#     manhattan_df = pd.read_excel(xlsx_path, sheet_name='Manhattan')
    
#     bronx_df['Date / Time'] = pd.to_datetime(bronx_df['Date / Time'])
#     manhattan_df['Date / Time'] = pd.to_datetime(manhattan_df['Date / Time'])
    
#     # Add normalized time column
#     bronx_df['Normalized Time'] = bronx_df['Date / Time'].apply(normalize_time_window)
#     manhattan_df['Normalized Time'] = manhattan_df['Date / Time'].apply(normalize_time_window)
    
#     bronx_df.sort_values('Normalized Time', inplace=True)
#     manhattan_df.sort_values('Normalized Time', inplace=True)
    
#     return bronx_df, manhattan_df

# def pick_station(lat, lon):
#     if 40.80 <= lat <= 40.92 and -73.93 <= lon <= -73.79:
#         return "Bronx"
#     else:
#         return "Manhattan"

# def match_nearest_weather(row_dt, weather_df):
#     # Use normalized time for matching
#     normalized_row_dt = normalize_time_window(row_dt)
#     diffs = (weather_df['Normalized Time'] - normalized_row_dt).abs()
#     idx_min = diffs.idxmin()
#     return weather_df.loc[idx_min]

# def add_weather_to_train(train_df, bronx_df, manh_df):
#     weather_cols = [
#         "Air Temp at Surface [degC]", 
#         "Relative Humidity [percent]", 
#         "Avg Wind Speed [m/s]", 
#         "Wind Direction [degrees]", 
#         "Solar Flux [W/m^2]"
#     ]
    
#     at_surface, rh, wind_speed, wind_dir, solar_flux = [], [], [], [], []
    
#     for _, row in train_df.iterrows():
#         stn = pick_station(row['Latitude'], row['Longitude'])
#         wdf = bronx_df if stn == 'Bronx' else manh_df
        
#         row_dt = pd.to_datetime(row['datetime'])
#         match = match_nearest_weather(row_dt, wdf)
        
#         at_surface.append(match["Air Temp at Surface [degC]"])
#         rh.append(match["Relative Humidity [percent]"])
#         wind_speed.append(match["Avg Wind Speed [m/s]"])
#         wind_dir.append(match["Wind Direction [degrees]"])
#         solar_flux.append(match["Solar Flux [W/m^2]"])
    
#     train_df["Weather_AirTemp"] = at_surface
#     train_df["Weather_RH"] = rh
#     train_df["Weather_WindSpd"] = wind_speed
#     train_df["Weather_WindDir"] = wind_dir
#     train_df["Weather_Solar"] = solar_flux
    
#     return train_df

# # Usage remains the same
# weather_path = '/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx'
# weather_bronx, weather_manh = load_weather_station(weather_path)
# train_enh = add_weather_to_train(train_df, weather_bronx, weather_manh)

In [22]:
# import pandas as pd
# import numpy as np

# # Example file paths (adjust as needed)
# weather_path = '/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx'
# # Suppose your training DataFrame is already loaded as train_df:
# # train_df = pd.read_csv('your_train_file.csv')

# # --- Step 1: Load and rename weather data ---

# def load_weather_station(xlsx_path):
#     # Load weather data from two sheets
#     bronx_df = pd.read_excel(xlsx_path, sheet_name='Bronx')
#     manhattan_df = pd.read_excel(xlsx_path, sheet_name='Manhattan')
    
#     # Convert the 'Date / Time' column to datetime if needed
#     bronx_df['Date / Time'] = pd.to_datetime(bronx_df['Date / Time'])
#     manhattan_df['Date / Time'] = pd.to_datetime(manhattan_df['Date / Time'])
    
#     # Rename columns using your provided dictionaries
#     bronx_column_names = {
#         bronx_df.columns[1]: 'surface_air_temp',
#         bronx_df.columns[2]: 'relative_humidity',
#         bronx_df.columns[3]: 'avg_wind_speed',
#         bronx_df.columns[4]: 'wind_direction',
#         bronx_df.columns[5]: 'solar_flux'
#     }
#     manhattan_column_names = {
#         manhattan_df.columns[1]: 'surface_air_temp',
#         manhattan_df.columns[2]: 'relative_humidity',
#         manhattan_df.columns[3]: 'avg_wind_speed',
#         manhattan_df.columns[4]: 'wind_direction',
#         manhattan_df.columns[5]: 'solar_flux'
#     }
#     bronx_df.rename(columns=bronx_column_names, inplace=True)
#     manhattan_df.rename(columns=manhattan_column_names, inplace=True)
    
#     # (Optional) sort by time if needed
#     bronx_df.sort_values('Date / Time', inplace=True)
#     manhattan_df.sort_values('Date / Time', inplace=True)
    
#     return bronx_df, manhattan_df

# weather_bronx, weather_manh = load_weather_station(weather_path)

# # --- Step 2: Define a function to pick station based on coordinates ---
# def pick_station(lat, lon):
#     # Simple example: if within Bronx bounds, return 'Bronx', else 'Manhattan'
#     if 40.80 <= lat <= 40.92 and -73.93 <= lon <= -73.79:
#         return "Bronx"
#     else:
#         return "Manhattan"

# # --- Step 3: Impute weather features into train_df using fixed chunks ---
# def add_weather_to_train(train_df, bronx_df, manhattan_df):
#     # Ensure a copy and convert datetime column to datetime type, then sort
#     train_df = train_df.copy()
#     train_df['datetime'] = pd.to_datetime(train_df['datetime'])
#     train_df.sort_values('datetime', inplace=True)
    
#     weather_features = ['surface_air_temp', 'relative_humidity', 'avg_wind_speed', 'wind_direction', 'solar_flux']
    
#     # Create empty columns for weather features
#     for feat in weather_features:
#         train_df[feat] = np.nan

#     # Determine station for each training row
#     train_df['station'] = train_df.apply(lambda row: pick_station(row['Latitude'], row['Longitude']), axis=1)
    
#     # Process Bronx rows: assign each chunk of train rows the corresponding weather row
#     bronx_train = train_df[train_df['station'] == 'Bronx'].copy()
#     bronx_train.sort_values('datetime', inplace=True)
#     chunk_size_bronx = 31
#     n_chunks_bronx = int(np.ceil(len(bronx_train) / chunk_size_bronx))
#     for i in range(n_chunks_bronx):
#         start = i * chunk_size_bronx
#         end = min((i + 1) * chunk_size_bronx, len(bronx_train))
#         weather_idx = i if i < len(bronx_df) else len(bronx_df) - 1
#         weather_row = bronx_df.iloc[weather_idx]
#         train_df.loc[bronx_train.index[start:end], weather_features] = weather_row[weather_features].values

#     # Process Manhattan rows similarly
#     manh_train = train_df[train_df['station'] == 'Manhattan'].copy()
#     manh_train.sort_values('datetime', inplace=True)
#     chunk_size_manh = 35
#     n_chunks_manh = int(np.ceil(len(manh_train) / chunk_size_manh))
#     for i in range(n_chunks_manh):
#         start = i * chunk_size_manh
#         end = min((i + 1) * chunk_size_manh, len(manh_train))
#         weather_idx = i if i < len(manhattan_df) else len(manhattan_df) - 1
#         weather_row = manhattan_df.iloc[weather_idx]
#         train_df.loc[manh_train.index[start:end], weather_features] = weather_row[weather_features].values

#     return train_df

# # --- Example usage ---
# # Suppose train_df is your training DataFrame (make sure it has columns 'datetime', 'Latitude', 'Longitude', etc.)
# # For example, if you load it from a CSV:
# # train_df = pd.read_csv('/kaggle/input/your_train.csv')

# # Now, add the weather features to train_df
# weather_path = '/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx'
# weather_bronx, weather_manh = load_weather_station(weather_path)
# train_enh = add_weather_to_train(train_df, weather_bronx, weather_manh)
# print(train_enh.head())


In [23]:
# train_enh.nunique()

In [24]:
# for _, row in train_df.iterrows():
#         stn = pick_station(row['Latitude'], row['Longitude'])
#         if stn == 'Bronx':
#             wdf = weather_bronx
#         else:
#             wdf = weather_manh

In [25]:
# train_df.loc[train_df['borough'] == 'Bronx']['borough']

In [26]:
# def merge_features(df, bronx_feats, manh_feats):
#     """Merge processed weather features based on location"""
#     df['borough'] = df.apply(lambda x: pick_station(x['Latitude'], x['Longitude']), axis=1)
    
#     # Create borough feature matrix
#     weather_data = pd.DataFrame({
#         'Bronx': bronx_feats,
#         'Manhattan': manh_feats
#     }).T
    
#     return df.merge(weather_data, left_on='borough', right_index=True)

# train_enh = merge_features(train_enh, bronx_features, manh_features)
# test_enh = merge_features(test_df.copy(), bronx_features, manh_features)

In [27]:
# from sklearn.neighbors import NearestNeighbors

# def enhanced_spatial_impute(train_df, test_df, feature_cols):
#     """Weighted spatial imputation using Haversine distance"""
#     # Convert to radians for haversine metric
#     train_rad = np.deg2rad(train_df[['Latitude', 'Longitude']])
#     test_rad = np.deg2rad(test_df[['Latitude', 'Longitude']])
    
#     # Find 10 nearest neighbors
#     nn = NearestNeighbors(n_neighbors=10, metric='haversine')
#     nn.fit(train_rad)
#     distances, indices = nn.kneighbors(test_rad)
    
#     # Inverse distance weighting
#     weights = 1 / (distances + 1e-6)
#     weights /= weights.sum(axis=1)[:, None]
    
#     # Impute features
#     for col in feature_cols:
#         test_df[col] = (train_df.iloc[indices.flatten()][col].values.reshape(indices.shape) * weights).sum(axis=1)
    
#     return test_df

# # Impute weather features
# weather_features = ['diurnal_temp_range', 'morning_warming_rate', 'thermal_inertia']
# test_enh = enhanced_spatial_impute(train_enh, test_enh, weather_features)


In [28]:
# import pandas as pd
# import numpy as np
# from datetime import datetime
# import math

# # -----------------------------
# # 1. Load Weather Data from XLSX
# # -----------------------------
# def load_weather_stations(xlsx_path):
#     """
#     Loads the weather data from two sheets (Bronx and Manhattan) from an XLSX file.
#     Returns two DataFrames.
#     """
#     bronx_df = pd.read_excel(xlsx_path, sheet_name="Bronx")
#     manh_df   = pd.read_excel(xlsx_path, sheet_name="Manhattan")
    
#     # Convert 'Date / Time' to datetime objects
#     bronx_df["Date / Time"] = pd.to_datetime(bronx_df["Date / Time"])
#     manh_df["Date / Time"]   = pd.to_datetime(manh_df["Date / Time"])
    
#     return bronx_df, manh_df

# # -----------------------------
# # 2. Aggregate Weather with Time Weighting
# # -----------------------------
# def weighted_weather_aggregate(weather_df, target_time_str="15:30:00", bandwidth=30):
#     """
#     Given a weather DataFrame (with 'Date / Time'), compute a weighted average
#     for each weather variable using a Gaussian weight on the minutes difference
#     from target_time (default 15:30).
    
#     bandwidth is in minutes.
#     Returns a dictionary of aggregated weather values.
#     """
#     # Convert target_time_str to a datetime.time object
#     target_time = datetime.strptime(target_time_str, "%H:%M:%S").time()
    
#     # For each row, compute time difference (in minutes) from target_time
#     def time_diff_minutes(dt):
#         # dt is a Timestamp; extract time and compute difference in minutes from target_time.
#         t = dt.time()
#         # Convert times to minutes from midnight:
#         minutes = t.hour * 60 + t.minute
#         target_minutes = target_time.hour * 60 + target_time.minute
#         return abs(minutes - target_minutes)
    
#     weather_df["time_diff"] = weather_df["Date / Time"].apply(time_diff_minutes)
#     # Compute Gaussian weights: weight = exp( - (diff/bandwidth)^2 )
#     weather_df["weight"] = np.exp(-(weather_df["time_diff"] / bandwidth) ** 2)
    
#     # List of weather variables to aggregate (adjust column names as needed)
#     vars_to_agg = ["Air Temp at Surface [degC]", "Relative Humidity [percent]",
#                      "Avg Wind Speed [m/s]", "Wind Direction [degrees]", "Solar Flux [W/m^2]"]
    
#     agg_dict = {}
#     for var in vars_to_agg:
#         # Weighted average: sum(var*weight) / sum(weight)
#         weighted_avg = (weather_df[var] * weather_df["weight"]).sum() / weather_df["weight"].sum()
#         agg_dict[var] = weighted_avg
        
#         # Also add standard deviation weighted (optional)
#         # weighted_std = np.sqrt(((weather_df[var] - weighted_avg)**2 * weather_df["weight"]).sum() / weather_df["weight"].sum())
#         # agg_dict[var + " (std)"] = weighted_std

#     return agg_dict

# # -----------------------------
# # 3. Robust Station Assignment Using Coordinates
# # -----------------------------
# def assign_station(lat, lon, station_centroids):
#     """
#     Given a latitude and longitude, assign the station based on nearest centroid.
#     station_centroids is a dict, e.g., {"Bronx": (lat1, lon1), "Manhattan": (lat2, lon2)}.
#     Returns the station name.
#     """
#     def haversine(lat1, lon1, lat2, lon2):
#         # Calculate the great circle distance between two points on Earth (in km)
#         R = 6371  # km
#         dlat = math.radians(lat2 - lat1)
#         dlon = math.radians(lon2 - lon1)
#         a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2)**2
#         c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
#         return R * c

#     best_station = None
#     best_dist = float("inf")
#     for stn, (stn_lat, stn_lon) in station_centroids.items():
#         d = haversine(lat, lon, stn_lat, stn_lon)
#         if d < best_dist:
#             best_dist = d
#             best_station = stn
#     return best_station

# # -----------------------------
# # 4. Merge Weather Features into Train and Test
# # -----------------------------
# def merge_weather_features(train_df, test_df, weather_xlsx_path):
#     """
#     - Loads weather data from the XLSX file (two sheets: Bronx and Manhattan).
#     - Aggregates weather using a Gaussian weight centered at 15:30.
#     - Uses robust station assignment (via nearest station centroid) to assign weather features.
#     - For train, use the actual datetime to get the weighted aggregate; for test, use nearest spatial assignment.
    
#     Returns updated train_df and test_df with new weather columns.
#     """
#     # Load weather data
#     bronx_df, manh_df = load_weather_stations(weather_xlsx_path)
#     # Aggregate weather for each station using the entire day but weighted around 15:30
#     bronx_agg = weighted_weather_aggregate(bronx_df, target_time_str="15:30:00", bandwidth=30)
#     manh_agg  = weighted_weather_aggregate(manh_df, target_time_str="15:30:00", bandwidth=30)
    
#     # Define station centroids for robust assignment (example coordinates; adjust as needed)
#     station_centroids = {
#         "Bronx": (40.84, -73.87),
#         "Manhattan": (40.75, -73.98)
#     }
    
#     # For train data: we have datetime, so we can average over the minute differences if desired.
#     # Here, we'll simply assign the station based on coordinates.
#     def assign_and_get_weather(row):
#         stn = assign_station(row["Latitude"], row["Longitude"], station_centroids)
#         row["station"] = stn
#         if stn == "Bronx":
#             return pd.Series(bronx_agg)
#         else:
#             return pd.Series(manh_agg)
    
#     # For train, apply for each row.
#     train_weather = train_df.apply(assign_and_get_weather, axis=1)
#     train_df = pd.concat([train_df, train_weather], axis=1)
    
#     # For test, if no datetime is provided, do the same based on coordinates.
#     test_weather = test_df.apply(assign_and_get_weather, axis=1)
#     test_df = pd.concat([test_df, test_weather], axis=1)
    
#     # Remove temporary station column if desired.
#     # train_df.drop(columns="station", inplace=True)
#     # test_df.drop(columns="station", inplace=True)
    
#     return train_df, test_df


# # Merge weather features using the full weather dataset
# train_df, test_df = merge_weather_features(train_df, test_df, weather_path)

# # Save the new CSV files
# train_df.to_csv("train_with_weather.csv", index=False)
# test_df.to_csv("test_with_weather.csv", index=False)

# print("Weather features merged successfully.")


# Approach_3

In [29]:
# import pandas as pd
# import numpy as np
# from sklearn.decomposition import PCA

# def create_station_pca_features(weather_df, n_components=3):
#     """
#     1) For each row in weather_df (times from 6:00–20:00), we have columns:
#        'Date / Time', AirTemp, RH, WindSpd, etc.
#     2) Extract the time-series as a matrix, e.g. shape=(num_timesteps, num_vars).
#     3) Perform PCA, keep n_components. 
#     4) Return the PCA scores as a single feature vector for that station.
#     """
#     # Sort by time
#     weather_df = weather_df.sort_values("Date / Time").reset_index(drop=True)
#     # Remove the datetime col for PCA
#     numeric_cols = ["Air Temp at Surface [degC]", "Relative Humidity [percent]",
#                     "Avg Wind Speed [m/s]", "Wind Direction [degrees]", "Solar Flux [W/m^2]"]
#     # shape => (num_timesteps, num_vars)
#     X = weather_df[numeric_cols].values
#     # If missing data, fill with mean or zero
#     X = np.nan_to_num(X, nan=np.nanmean(X, axis=0))
    
#     pca = PCA(n_components=n_components)
#     scores = pca.fit_transform(X.T)  # or X, depending on dimension you want
#     # shape => (num_vars, n_components) if we do X.T 
#     # or (num_timesteps, n_components) if we do X

#     # We'll flatten or pick the first principal components in a consistent shape
#     # E.g. flatten scores:
#     feature_vec = scores.flatten()
#     return feature_vec, pca.explained_variance_ratio_

# def approach_1_station_features(bronx_df, manh_df):
#     """
#     Returns a dict with station: feature_vector from PCA across the entire day.
#     """
#     brx_vec, brx_var = create_station_pca_features(bronx_df, n_components=3)
#     mnh_vec, mnh_var = create_station_pca_features(manh_df, n_components=3)
#     station_features = {
#         "Bronx": brx_vec,
#         "Manhattan": mnh_vec
#     }
#     return station_features

# def assign_station(lat, lon):
#     """
#     Minimal bounding or logic to pick 'Bronx' vs. 'Manhattan'.
#     Replace with your actual approach.
#     """
#     return "Bronx" if lat>40.82 else "Manhattan"

# def apply_approach_1(train_df, test_df, bronx_df, manh_df):
#     """
#     - Creates PCA-based feature vectors for each station from the entire day.
#     - Merges them into train/test by station assignment.
#     """
#     station_feats = approach_1_station_features(bronx_df, manh_df)
#     n_pca = len(station_feats["Bronx"])  # e.g. 15 if we did n_components=3 over 5 vars, depends on shape

#     # Add columns
#     col_names = [f"WeatherPCA_{i}" for i in range(n_pca)]

#     def get_features_for_station(row):
#         stn = assign_station(row["Latitude"], row["Longitude"])
#         vec = station_feats[stn]
#         return pd.Series(vec)

#     train_df[col_names] = train_df.apply(get_features_for_station, axis=1)
#     test_df[col_names]  = test_df.apply(get_features_for_station, axis=1)
#     return train_df, test_df

# train_enh, test_enh = apply_approach_1(train_df, test_df, bronx_df, manhattan_df)

# Approach_4

In [30]:
# import pandas as pd
# import numpy as np
# from scipy.interpolate import interp1d

# # Simplified coordinate ranges for Bronx and Manhattan
# BRONX_BOUNDS = {'lat_min': 40.7855, 'lat_max': 40.9155, 'lon_min': -73.9332, 'lon_max': -73.7654}
# MANHATTAN_BOUNDS = {'lat_min': 40.6990, 'lat_max': 40.8821, 'lon_min': -74.0256, 'lon_max': -73.9070}

# def assign_borough(lat, lon):
#     if (BRONX_BOUNDS['lat_min'] <= lat <= BRONX_BOUNDS['lat_max']) and \
#        (BRONX_BOUNDS['lon_min'] <= lon <= BRONX_BOUNDS['lon_max']):
#         return 'Bronx'
#     elif (MANHATTAN_BOUNDS['lat_min'] <= lat <= MANHATTAN_BOUNDS['lat_max']) and \
#          (MANHATTAN_BOUNDS['lon_min'] <= lon <= MANHATTAN_BOUNDS['lon_max']):
#         return 'Manhattan'
#     return 'Unknown'

# def preprocess_weather(weather_df):
#     weather_df['Date / Time'] = pd.to_datetime(weather_df['Date / Time']).dt.tz_localize(None)
#     return weather_df.set_index('Date / Time')

# def impute_weather_data(train_df, test_df, bronx_weather, manhattan_weather):
#     train_df['borough'] = train_df.apply(lambda x: assign_borough(x['Latitude'], x['Longitude']), axis=1)
#     test_df['borough'] = test_df.apply(lambda x: assign_borough(x['Latitude'], x['Longitude']), axis=1)

#     # Preprocess weather data
#     bronx_weather = preprocess_weather(bronx_weather)
#     manhattan_weather = preprocess_weather(manhattan_weather)
#     weather_dfs = {'Bronx': bronx_weather, 'Manhattan': manhattan_weather}

#     # Interpolation for train data
#     train_weather = []
#     for borough in ['Bronx', 'Manhattan']:
#         borough_train = train_df[train_df['borough'] == borough].copy()
#         weather = weather_dfs[borough]

#         time_numeric = weather.index.astype(np.int64) // 10**9
#         borough_times = pd.to_datetime(borough_train['datetime']).astype(np.int64) // 10**9

#         interpolators = {
#             col: interp1d(time_numeric, weather[col], bounds_error=False, fill_value='extrapolate')
#             for col in weather.columns
#         }

#         for col, interp_func in interpolators.items():
#             borough_train[f'weather_{col}'] = interp_func(borough_times)

#         train_weather.append(borough_train)

#     train_imputed = pd.concat(train_weather)

#     # Proximity-based interpolation for test data
#     test_weather = []
#     for borough in ['Bronx', 'Manhattan']:
#         borough_test = test_df[test_df['borough'] == borough].copy()
#         borough_train = train_imputed[train_imputed['borough'] == borough]

#         # Skip if borough_train is empty
#         if borough_train.empty:
#             continue

#         for col in [c for c in train_imputed.columns if c.startswith('weather_')]:
#             def find_nearest_weather(row):
#                 distances = ((borough_train['Latitude'] - row['Latitude'])**2 +
#                              (borough_train['Longitude'] - row['Longitude'])**2)
#                 if distances.isnull().all():
#                     return np.nan  # Handle case where all distances are NaN
#                 nearest_idx = distances.idxmin()
#                 return borough_train.loc[nearest_idx, col] if nearest_idx in borough_train.index else np.nan

#             borough_test[f'{col}_imputed'] = borough_test.apply(find_nearest_weather, axis=1)

#         test_weather.append(borough_test)

#     test_imputed = pd.concat(test_weather)

#     return train_imputed, test_imputed

# # Example usage
# train_processed, test_processed = impute_weather_data(
#     train_df=train_df,
#     test_df=test_df,
#     bronx_weather=bronx_df,
#     manhattan_weather=manhattan_df
# )

# # Verify results
# print("Train weather columns:", [c for c in train_processed if c.startswith('weather_')])
# print("Test weather values sample:\n", test_processed[['borough', 'weather_Air Temp at Surface [degC]_imputed']].head())
# train_processed.head(3)

In [31]:
# new_column_names = {
#     train_processed.columns[5]: 'surface_air_temp',
#     train_processed.columns[6]: 'relative_humidity',
#     train_processed.columns[7]: 'avg_wind_speed',
#     train_processed.columns[8]: 'wind_direction',
#     train_processed.columns[9]: 'solar_flux'
# }
# test_column_names = {
#     test_processed.columns[4]: 'surface_air_temp',
#     test_processed.columns[5]: 'relative_humidity',
#     test_processed.columns[6]: 'avg_wind_speed',
#     test_processed.columns[7]: 'wind_direction',
#     test_processed.columns[8]: 'solar_flux'
# }
# train_processed.rename(columns=new_column_names, inplace=True)
# test_processed.rename(columns=test_column_names, inplace=True)

# Approach_5

In [32]:
# import pandas as pd
# import numpy as np
# import geopandas as gpd
# from shapely.geometry import Point

# # ======================================================================
# # 1. Borough Assignment with Proper Column Names
# # ======================================================================

# def assign_borough(lat, lon):
#     """Improved borough assignment using actual coordinate columns"""
#     # Simplified threshold - adjust based on your coordinate analysis
#     BRONX_NORTHERN_BOUNDARY = 40.87
#     return "Bronx" if lat > BRONX_NORTHERN_BOUNDARY else "Manhattan"

# # Apply to datasets with correct Latitude/Longitude columns
# train_df['borough'] = train_df.apply(
#     lambda x: assign_borough(x['Latitude'], x['Longitude']), axis=1
# )
# test_df['borough'] = test_df.apply(
#     lambda x: assign_borough(x['Latitude'], x['Longitude']), axis=1
# )

# # ======================================================================
# # 2. Weather Feature Engineering with Correct Column Names
# # ======================================================================

# def process_weather(weather_df):
#     """Enhanced temporal feature extraction with proper column references"""
#     # Ensure the index is a DatetimeIndex
#     weather_df.index = pd.to_datetime(weather_df.index, errors='coerce')
    
#     # Filter data between 3 PM and 4 PM
#     weather_3pm = weather_df.between_time('15:00', '16:00').copy()
    
#     return pd.Series({
#         'temp_slope': np.polyfit(
#             np.arange(len(weather_3pm)), 
#             weather_3pm['surface_air_temp'], 
#             1
#         )[0] if len(weather_3pm) > 1 else 0,
#         'humidity_var': weather_3pm['relative_humidity'].var() if not weather_3pm['relative_humidity'].isna().all() else 0,
#         'wind_dir_mode': weather_3pm['wind_direction'].mode()[0] if not weather_3pm['wind_direction'].isna().all() else 0,
#         'solar_cumulative': weather_3pm['solar_flux'].sum(),
#         'wind_speed_max': weather_3pm['avg_wind_speed'].max() if not weather_3pm['avg_wind_speed'].isna().all() else 0
#     })

# # Process both boroughs
# bronx_features = process_weather(
#     bronx_df.rename(columns={'Date / Time': 'Date/Time'}).set_index('Date/Time')
# )
# manhattan_features = process_weather(
#     manhattan_df.rename(columns={'Date / Time': 'Date/Time'}).set_index('Date/Time')
# )

# # ======================================================================
# # 3. Spatial Feature Integration
# # ======================================================================

# # Create geometries first
# train_df['geometry'] = train_df.apply(
#     lambda x: Point(x['Longitude'], x['Latitude']), 
#     axis=1
# )
# test_df['geometry'] = test_df.apply(
#     lambda x: Point(x['Longitude'], x['Latitude']), 
#     axis=1
# )

# def calculate_density(row):
#     """Building density with 100m buffer"""
#     buffer = row['geometry'].buffer(0.0009)  # ~100m at NYC latitude
#     return len(building_density[building_density.intersects(buffer)])

# # Apply density calculation
# # train_df['building_density'] = train_df.apply(calculate_density, axis=1)
# # test_df['building_density'] = test_df.apply(calculate_density, axis=1)

# # # ======================================================================
# # # 4. Final Feature Merging
# # # ======================================================================

# # Create feature matrix
# weather_features = pd.DataFrame({
#     'Bronx': bronx_features,
#     'Manhattan': manhattan_features
# }).T.add_prefix('weather_')

# # Merge with main data
# train_df = train_df.merge(
#     weather_features,
#     left_on='borough', 
#     right_index=True
# )
# test_df = test_df.merge(
#     weather_features,
#     left_on='borough', 
#     right_index=True
# )

# # ======================================================================
# # 5. Critical Enhancements for Model Performance
# # ======================================================================

# # Add interaction features
# train_df['temp_wind_interaction'] = (
#     train_df['weather_temp_slope'] * 
#     train_df['weather_wind_speed_max']
# )
# test_df['temp_wind_interaction'] = (
#     test_df['weather_temp_slope'] * 
#     test_df['weather_wind_speed_max']
# )

# # # Temporal alignment feature
# # train_df['minute_offset'] = (
# #     train_df['datetime'].dt.minute - 180
# # )  # Minutes since 3:00 PM

In [33]:
weather_cols = ['Weather_AirTemp',
       'Weather_RH', 'Weather_WindSpd', 'Weather_WindDir', 'Weather_Solar']

In [34]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors

# ------------------------------
# Weather Processing Functions
# ------------------------------

def load_weather_station(xlsx_path, target_lower, target_upper):
    """
    Load weather data from an Excel file and compress its time information to a target time window.
    
    Parameters:
      - xlsx_path: Path to the weather Excel file.
      - target_lower: The lower bound of the target time window (e.g., 10 for 10:00).
      - target_upper: The upper bound of the target time window (e.g., 18 for 18:00).
    
    The function:
      1. Loads the Bronx and Manhattan sheets.
      2. Processes each DataFrame:
         - Converts 'Date / Time' to datetime.
         - Filters rows to only those within the target window (e.g., 10:00 to 18:00).
         - Converts the time to decimal hours.
         - Computes the observed minimum and maximum time (orig_lower, orig_upper) in the filtered data.
         - Linearly maps (compresses) the observed time range to exactly [target_lower, target_upper].
         - Replaces 'Date / Time' with the normalized date plus the compressed time.
    
    Returns:
      A tuple (bronx_df, manhattan_df) of the processed DataFrames.
    """
    # Load sheets
    bronx_df = pd.read_excel(xlsx_path, sheet_name='Bronx')
    manhattan_df = pd.read_excel(xlsx_path, sheet_name='Manhattan')
    
    def process_and_compress(df):
        # Convert 'Date / Time' to datetime
        df['Date / Time'] = pd.to_datetime(df['Date / Time'])
        
        # Filter rows to only include records in the target window (e.g., between target_lower and target_upper)
        df['hour'] = df['Date / Time'].dt.hour
        df = df[(df['hour'] >= target_lower) & (df['hour'] < target_upper)]
        
        # Convert the time portion to decimal hours
        df['time_hours'] = (df['Date / Time'].dt.hour +
                            df['Date / Time'].dt.minute / 60.0 +
                            df['Date / Time'].dt.second / 3600.0)
        
        # Determine the observed min and max in the filtered data
        orig_lower, orig_upper = df['time_hours'].min(), df['time_hours'].max()
        # Set the target window as provided
        new_lower, new_upper = target_lower, target_upper
        
        # Compress (scale) the observed time range to the target window.
        # If orig_lower == new_lower and orig_upper == new_upper, this will be an identity mapping.
        df['compressed_time'] = new_lower + (df['time_hours'] - orig_lower) * ((new_upper - new_lower) / (orig_upper - orig_lower))
        
        # Replace 'Date / Time' with the normalized date plus the compressed time
        df['Date / Time'] = df['Date / Time'].dt.normalize() + pd.to_timedelta(df['compressed_time'], unit='h')
        
        # Sort and clean up intermediate columns
        df.sort_values('Date / Time', inplace=True)
        df.drop(columns=['hour', 'time_hours', 'compressed_time'], inplace=True)
        return df

    # Process both sheets
    bronx_df = process_and_compress(bronx_df)
    manhattan_df = process_and_compress(manhattan_df)
    return bronx_df, manhattan_df

def pick_station(lat, lon):
    """Select the weather station based on latitude and longitude."""
    if 40.80 <= lat <= 40.92 and -73.93 <= lon <= -73.79:
        return "Bronx"
    else:
        return "Manhattan"

def get_closest_weather(target_dt, weather_df):
    """
    Return weather features from the row with the closest datetime.
    The output dictionary prefixes keys with 'Weather_'.
    """
    same_day = weather_df[weather_df['Date / Time'].dt.date == target_dt.date()]
    if same_day.empty:
        return None
    diffs = (same_day['Date / Time'] - target_dt).abs()
    idx_min = diffs.idxmin()
    row = same_day.loc[idx_min]
    return {
        'Weather_AirTemp': row['Air Temp at Surface [degC]'],
        'Weather_RH': row['Relative Humidity [percent]'],
        'Weather_WindSpd': row['Avg Wind Speed [m/s]'],
        'Weather_WindDir': row['Wind Direction [degrees]'],
        'Weather_Solar': row['Solar Flux [W/m^2]']
    }

def add_weather_to_train(train_df, bronx_df, manh_df):
    """
    Match each training row to the closest weather row (by datetime) and attach weather features.
    """
    # Group weather data by date for quicker lookup
    bronx_dates = {d: g for d, g in bronx_df.groupby(bronx_df['Date / Time'].dt.date)}
    manh_dates = {d: g for d, g in manh_df.groupby(manh_df['Date / Time'].dt.date)}
    
    weather_cols = ['Weather_AirTemp', 'Weather_RH', 'Weather_WindSpd', 'Weather_WindDir', 'Weather_Solar']
    for col in weather_cols:
        train_df[col] = np.nan
    
    train_df['datetime'] = pd.to_datetime(train_df['datetime'])
    for idx, row in train_df.iterrows():
        stn = pick_station(row['Latitude'], row['Longitude'])
        target_date = row['datetime'].date()
        weather_data = bronx_dates.get(target_date) if stn == 'Bronx' else manh_dates.get(target_date)
        if weather_data is not None:
            features = get_closest_weather(row['datetime'], weather_data)
            if features is not None:
                for col in weather_cols:
                    train_df.at[idx, col] = features[col]
    return train_df

# ------------------------------
# Spatial Imputation Functions
# ------------------------------

def df_to_gdf(df, lon_col='Longitude', lat_col='Latitude', crs='EPSG:4326'):
    """
    Convert a DataFrame to a GeoDataFrame with Point geometry.
    """
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    return gpd.GeoDataFrame(df, geometry=geometry, crs=crs)

def impute_weather_from_train(train_gdf, test_gdf, weather_cols):
    """
    Impute weather features for the test set using geopandas' sjoin_nearest.
    Assumes train_gdf contains weather_cols and geometry.
    """
    train_subset = train_gdf[weather_cols + ['geometry']]
    joined = gpd.sjoin_nearest(test_gdf, train_subset, how="left", distance_col="distance")
    joined = joined.drop(columns=['index_right', 'distance'])
    return joined

# ------------------------------
# Integrated Processing Function
# ------------------------------

def process_time_variants_and_impute_test(train_df, test_df, weather_path, time_variants, weather_cols):
    """
    For each time variant (a tuple of (target_lower, target_upper)):
      1. Process the weather data from the Excel file and enhance training data by matching weather features.
      2. Convert train and test DataFrames to GeoDataFrames and reproject them.
      3. Impute weather features for the test set using spatial nearest neighbor matching.
      4. Save the enhanced training and imputed test sets to CSV.
    
    Parameters:
      - train_df: Training DataFrame (must contain 'datetime', 'Latitude', 'Longitude').
      - test_df: Test DataFrame (must contain 'Latitude', 'Longitude').
      - weather_path: Path to the weather Excel file.
      - time_variants: List of tuples, each (target_lower, target_upper). For example, [(10,18), (11,17)].
      - weather_cols: List of weather feature column names.
    """
    for target_lower, target_upper in time_variants:
        print(f"\nProcessing time variant with target window [{target_lower}, {target_upper})...")
        
        # Process weather data and enhance training data using the dynamic target window.
        weather_bronx, weather_manh = load_weather_station(weather_path, target_lower, target_upper)
        train_enh = add_weather_to_train(train_df.copy(), weather_bronx, weather_manh)
        train_csv = f"train_enh_{target_lower}_{target_upper}.csv"
        train_enh.to_csv(train_csv, index=False)
        print(f"Saved enhanced training data: {train_csv}")
        
        # Convert DataFrames to GeoDataFrames
        train_gdf = df_to_gdf(train_enh, lon_col='Longitude', lat_col='Latitude')
        test_gdf = df_to_gdf(test_df.copy(), lon_col='Longitude', lat_col='Latitude')
        
        # Reproject to a common CRS (e.g., EPSG:2263) for accurate spatial operations
        train_gdf = train_gdf.to_crs('EPSG:2263')
        test_gdf = test_gdf.to_crs('EPSG:2263')
        
        # Impute weather features for the test set via nearest spatial neighbor matching
        test_gdf = impute_weather_from_train(train_gdf, test_gdf, weather_cols)
        
        # Reproject test set back to the original CRS (EPSG:4326)
        test_gdf = test_gdf.to_crs('EPSG:4326')
        test_csv = f"test_imputed_{target_lower}_{target_upper}.csv"
        test_gdf.to_csv(test_csv, index=False)
        print(f"Saved imputed test data: {test_csv}")

# ------------------------------
# Example usage:
# ------------------------------

# Load your train and test DataFrames as needed:
# train_df = pd.read_csv('path_to_train.csv')
# test_df = pd.read_csv('path_to_test.csv')

# Define the time variants you want to test (adjust the values as necessary)
time_variants_1 = [
    (11,20),
    (11,19),
    (12, 17),
    (13, 17),
    (14, 17),
    (14,20),
    (13,20),
    (14,18),
    (14,19),
    (11,17),
    (11,18),
    (6,20),
    (9,20),
]
time_variants_2 = [
    (11.30,17.30), 
    (10,18),
   (10.30, 18),
    (10, 18.30),
    (10.30, 18.30),
    (11,18),
    (11.30,18),
    (11.30,18.30),
    (11,18.30),
    (11.30,19),
    (11.30,19.30),
    (11.30,16.30),
    (11,15),
]


# Define the weather columns to be imputed (should match those added in add_weather_to_train)
weather_cols = ['Weather_AirTemp', 'Weather_RH', 'Weather_WindSpd', 'Weather_WindDir', 'Weather_Solar']

# Run the integrated processing function
process_time_variants_and_impute_test(train_df, test_df, '/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx', time_variants_1, weather_cols)
process_time_variants_and_impute_test(train_df, test_df, '/kaggle/input/ey-open-science-training-data/NY_Mesonet_Weather.xlsx', time_variants_2, weather_cols)


Processing time variant with target window [11.3, 17.3)...
Saved enhanced training data: train_enh_11.3_17.3.csv
Saved imputed test data: test_imputed_11.3_17.3.csv

Processing time variant with target window [10, 18)...
Saved enhanced training data: train_enh_10_18.csv
Saved imputed test data: test_imputed_10_18.csv

Processing time variant with target window [10.3, 18)...
Saved enhanced training data: train_enh_10.3_18.csv
Saved imputed test data: test_imputed_10.3_18.csv

Processing time variant with target window [10, 18.3)...
Saved enhanced training data: train_enh_10_18.3.csv
Saved imputed test data: test_imputed_10_18.3.csv

Processing time variant with target window [10.3, 18.3)...
Saved enhanced training data: train_enh_10.3_18.3.csv
Saved imputed test data: test_imputed_10.3_18.3.csv

Processing time variant with target window [11, 18)...
Saved enhanced training data: train_enh_11_18.csv
Saved imputed test data: test_imputed_11_18.csv

Processing time variant with target win

# imputing test with the help of train with spacial approximity

In [35]:
# check_df = pd.read_csv('/kaggle/working/test_imputed_11.3_17.3.csv')
# check_df.nunique()

Longitude          1032
Latitude           1030
UHI Index             0
geometry           1040
Weather_AirTemp       8
Weather_RH           18
Weather_WindSpd      16
Weather_WindDir      22
Weather_Solar        24
dtype: int64

In [36]:
# import geopandas as gpd
# from shapely.geometry import Point
# from sklearn.neighbors import NearestNeighbors

# def df_to_gdf(df, lon_col = 'Longitude', lat_col = 'Latitude', crs = 'EPSG:4326'):
#     geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
#     return gpd.GeoDataFrame(df, geometry=geometry, crs=crs)

# def impute_weather_from_train(train_gdf, test_gdf, weather_cols):
#     X_train = np.array(list(zip(train_gdf.geometry.x, train_gdf.geometry.y)))
#     X_test = np.array(list(zip(test_gdf.geometry.x, test_gdf.geometry.y)))

#     nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_train)
#     distances, indices = nbrs.kneighbors(X_test)

#     for col in weather_cols:
#         test_gdf[col] = train_gdf.iloc[indices.flatten()][col].values

#     return test_gdf
# # def impute_weather_from_train(train_gdf, test_gdf, weather_cols, k=3):
# #     X_train = np.array(list(zip(train_gdf.geometry.x, train_gdf.geometry.y)))
# #     X_test = np.array(list(zip(test_gdf.geometry.x, test_gdf.geometry.y)))

# #     nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(X_train)
# #     distances, indices = nbrs.kneighbors(X_test)

# #     # Inverse distance weighting
# #     weights = 1 / (distances + 1e-8)  # Adding small value to avoid division by zero
# #     weights /= weights.sum(axis=1, keepdims=True)

# #     for col in weather_cols:
# #         imputed_values = np.sum(train_gdf.iloc[indices.flatten()][col].values.reshape(-1, k) * weights, axis=1)
# #         test_gdf[col] = imputed_values

# #     return test_gdf

# train_gdf = df_to_gdf(train_enh_, lon_col='Longitude', lat_col='Latitude')
# test_gdf = df_to_gdf(test_df, lon_col='Longitude', lat_col='Latitude')

# train_gdf = train_gdf.to_crs('EPSG:2263')
# test_gdf = test_gdf.to_crs('EPSG:2263')

# # weather_cols = ['surface_air_temp', 'relative_humidity', 'avg_wind_speed', 'wind_direction', 'solar_flux']

# test_6_20 = impute_weather_from_train(train_gdf, test_gdf, weather_cols)
# test_gdf = test_gdf.to_crs('EPSG:4326')

In [37]:
# input_cols = [col for col in train_enh.columns if col!='Longitude' and col!='Latitude' and col!='UHI Index' and col!='geometry' and col!='datetime' and col!='time' and col!='borough' and col!='station']
# input_cols

In [38]:
# # weather_cols = [
# #     "Weather_AirTemp", "Weather_RH", "Weather_WindSpd",
# #     "Weather_WindDir", "Weather_Solar", 'THI', 'Temp_Solar_Interaction',
# #     'Wind_Temp_Interaction', 'Humidity_Solar_Interaction'
# # ]
# # def new_features(df):
# #     df['THI'] = df['Weather_AirTemp'] - (0.55 * (1 - df['Weather_RH'] / 100) * (df['Weather_AirTemp'] - 14.5))
# #     df['Temp_Solar_Interaction'] = df['Weather_AirTemp'] * df['Weather_Solar']
# #     df['Wind_Temp_Interaction'] = df['Weather_WindSpd'] * df['Weather_AirTemp']
# #     df['Humidity_Solar_Interaction'] = df['Weather_RH'] * df['Weather_Solar']
# #     return df

# train_weather = train_enh[input_cols]
# test_weather = test_gdf[input_cols]
# train_weather.to_csv('train_weather.csv', index=False)
# test_weather.to_csv('test_weather.csv', index=False)

In [39]:
# test_gdf.nunique()

In [40]:
# train = pd.read_csv('/kaggle/working/train_enh_12_17.csv')
# test = pd.read_csv('/kaggle/working/test_imputed_12_17.csv')
# train.drop(columns=['Longitude', 'Latitude', 'datetime', 'UHI Index', 'time'], inplace=True)
# test.drop(columns=['Longitude', 'Latitude', 'UHI Index', 'geometry'], inplace=True)

In [41]:
# import xgboost as xgb
# import lightgbm as lgb
# target = train_df['UHI Index']
# train[weather_cols] = train[weather_cols].astype('float64')
# test[weather_cols] = test[weather_cols].astype('float64')
# model = xgb.XGBRegressor(enable_categorical=False)
# model.fit(train_enh[weather_cols], target)

In [42]:
# preds = model.predict(test[weather_cols])
# preds

In [43]:
# subm_test = pd.read_csv('/kaggle/input/test-df-ey-open-science/Submission_template_UHI2025-v2.csv')
# subm_test['UHI Index'] = preds
# subm_test.to_csv('submission_weather.csv', index=False)

In [44]:
# train_enh.columns

In [45]:
# def filter_afternoon(df):
#         return df[
#             (df['Date / Time'].dt.hour==15) |
#             (df['Date / Time'].dt.hour==16)
#         ]
# bronx_3pm = filter_afternoon(bronx).mean(numeric_only=True).to_frame().T
# manhattan_3pm = filter_afternoon(manhattan).mean(numeric_only=True).to_frame().T
# manhattan_3pm

In [46]:
# def process_weather_data(weather_path):
#     bronx = pd.read_excel(weather_path, sheet_name='Bronx', parse_dates=['Date / Time'])
#     manhattan = pd.read_excel(weather_path, sheet_name='Manhattan', parse_dates=['Date / Time'])

#     def filter_afternoon(df):
#         return df[
#             (df['Date / Time'].dt.hour==15) |
#             (df['Date / Time'].dt.hour==16)
#         ]
#     bronx_3pm = filter_afternoon(bronx).mean(numeric_only=True).to_frame().T
#     manhattan_3pm = filter_afternoon(manhattan).mean(numeric_only=True).to_frame().T
#     bronx_3pm['borough'] = "Bronx"
#     manhattan_3pm['borough'] = 'Manhattan'

#     weather_agg = pd.concat([bronx_3pm, manhattan_3pm], ignore_index=True)
#     weather_agg.columns = [f'{col}_3pm' if col !='borough' else col for col in weather_agg.columns]
#     return weather_agg

In [47]:
# from shapely.geometry import Point

# def assign_borough(df, lon_col='Longitude', lat_col='Latitude'):
#     df = df.copy()
#     df['borough'] = 'Unknown'

#     bronx_bbox = (
#         (-73.9335 <= df[lon_col]) & (df[lon_col] <= -73.7654) *
#         (40.7855 <= df[lat_col]) & (df[lat_col] <= 40.9155)
#     )
#     manhattan_bbox = (
#         (-74.0500 <= df[lon_col]) & (df[lon_col] <= -73.9000) &
#         (40.7000 <= df[lat_col]) & (df[lat_col] <= 40.8800)
#     )

#     df.loc[bronx_bbox, 'borough'] = 'Bronx'
#     df.loc[manhattan_bbox, 'borough'] = "Manhattan"
#     return df

In [48]:
# weather_agg = process_weather_data(weather_path)

# train_wdf = assign_borough(train_df)
# test_wdf = assign_borough(test_df)

In [49]:
# train_merged = pd.merge(train_wdf, weather_agg, on='borough', how='left')
# test_merged = pd.merge(test_wdf, weather_agg, on='borough', how='left')

In [50]:
# input_cols = ['Air Temp at Surface [degC]_3pm', 'Relative Humidity [percent]_3pm',
#        'Avg Wind Speed [m/s]_3pm', 'Wind Direction [degrees]_3pm',
#        'Solar Flux [W/m^2]_3pm']

In [51]:
# # train_merged[input_cols].rename(columns={'Air Temp at Surface [degC]_3pm':'air_temp', 'Relative Humidity [percent]_3pm':'relative_humitdiy',
# #        'Avg Wind Speed [m/s]_3pm':'Avg_Wind_Speed', 'Wind Direction [degrees]_3pm':'wind_direction',
# #        'Solar Flux [W/m^2]_3pm':'Solar_Flux'}, inplace=True)
# train_merged[input_cols].rename(str, axis='columns', inplace=True)

In [52]:
# train_merged[input_cols] = train_merged[input_cols].astype('float64')
# test_merged[input_cols] = test_merged[input_cols].astype('float64')

In [53]:
# import xgboost as xgb
# train_merged.columns = train_merged.columns.str.replace(r'[\[\]<>]', '', regex=True)
# test_merged.columns = test_merged.columns.str.replace(r'[\[\]<>]', '', regex=True)
# # Prepare features and target
# input_cols = [col for col in train_merged.columns if col != 'UHI Index' and col != 'Latitude' and col!= 'Longitude' and col != 'datetime' and col!= 'time' and  col !='borough']
# print(input_cols)
# train_merged[input_cols] = train_merged[input_cols].astype('category')
# test_merged[input_cols] = test_merged[input_cols].astype('category')
# model = xgb.XGBRegressor()
# model.fit(train_merged[input_cols], target)